In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('obesity_data.csv')


In [3]:
df_train

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,Male,173.575262,71.982051,23.891783,4,Normal weight
1,69,Male,164.127306,89.959256,33.395209,2,Obese
2,46,Female,168.072202,72.930629,25.817737,4,Overweight
3,32,Male,168.459633,84.886912,29.912247,3,Overweight
4,60,Male,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,Male,155.588674,64.103182,26.480345,4,Overweight
996,35,Female,165.076490,97.639771,35.830783,1,Obese
997,49,Female,156.570956,78.804284,32.146036,1,Obese
998,64,Male,164.192222,57.978115,21.505965,4,Normal weight


In [4]:
# Codificar la variable categórica "Gender"
gender_encoder = LabelEncoder()
df_train['Gender'] = gender_encoder.fit_transform(df_train['Gender'])

In [5]:
df_train


,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,1,173.575262,71.982051,23.891783,4,Normal weight
1,69,1,164.127306,89.959256,33.395209,2,Obese
2,46,0,168.072202,72.930629,25.817737,4,Overweight
3,32,1,168.459633,84.886912,29.912247,3,Overweight
4,60,1,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,1,155.588674,64.103182,26.480345,4,Overweight
996,35,0,165.076490,97.639771,35.830783,1,Obese
997,49,0,156.570956,78.804284,32.146036,1,Obese
998,64,1,164.192222,57.978115,21.505965,4,Normal weight


In [7]:
features = df_train[['Age', 'Gender', 'Height', 'Weight', 'BMI', 'PhysicalActivityLevel']]

# Seleccionamos nuestra variable que vamos a predecir

labels = df_train['ObesityCategory']

In [8]:
# Codificar las etiquetas como números
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

In [10]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [11]:
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test)


In [24]:
X_train_tensor.shape

torch.Size([800, 6])

In [25]:
X_train_tensor.shape[1]

6

In [26]:
y_train_tensor.shape

torch.Size([800])

In [12]:
class ObesityModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ObesityModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

In [13]:
input_size = X_train_tensor.shape[1] ### Numero de caracteristicas
hidden_size = 64 
output_size = len(label_encoder.classes_) #Número de clases

model = ObesityModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss() # Lo utilizamos para un problema de clasificación
optimizer = optim.Adam(model.parameters(), lr = 0.001)


In [14]:
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    loss.backward()
    optimizer.step()



In [15]:
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Perdida del ensayo: {test_loss.item()}')


Perdida del ensayo: 0.7641244530677795


In [16]:
X = df_train.drop('ObesityCategory', axis = 1)

X.head()


,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel
0,56,1,173.575262,71.982051,23.891783,4
1,69,1,164.127306,89.959256,33.395209,2
2,46,0,168.072202,72.930629,25.817737,4
3,32,1,168.459633,84.886912,29.912247,3
4,60,1,183.568568,69.038945,20.487903,3


In [17]:
X_tensor = torch.FloatTensor(scaler.transform(X))


In [18]:
model.eval()

with torch.no_grad():
    predictions = model(X_tensor)


In [19]:
prediction_probs = torch.softmax(predictions, dim = 1)


In [20]:
predicted_classes = torch.argmax(prediction_probs, dim = 1)


In [27]:
predicted_classes

tensor([0, 1, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 2, 0, 0, 1, 0,
        3, 0, 0, 1, 2, 1, 2, 1, 0, 0, 2, 0, 0, 1, 1, 1, 3, 2, 1, 0, 0, 3, 1, 0,
        0, 0, 2, 0, 0, 2, 0, 3, 2, 2, 2, 0, 2, 1, 2, 0, 0, 0, 3, 1, 2, 1, 1, 0,
        2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0,
        0, 2, 1, 1, 2, 0, 0, 1, 0, 1, 2, 3, 0, 0, 0, 2, 0, 3, 2, 1, 0, 2, 2, 2,
        2, 1, 0, 3, 0, 0, 0, 1, 0, 0, 2, 0, 3, 2, 1, 3, 3, 1, 2, 1, 1, 0, 2, 2,
        1, 3, 1, 2, 1, 2, 3, 1, 0, 0, 2, 2, 1, 0, 1, 2, 0, 1, 2, 1, 2, 0, 0, 1,
        1, 0, 1, 2, 0, 0, 0, 1, 0, 3, 2, 1, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 3, 0,
        0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 1, 2, 3, 2, 0, 0, 0, 3, 0, 2,
        0, 1, 0, 3, 0, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 0, 2, 2, 0, 1, 2, 0, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 2, 2, 1, 2, 0, 0, 0, 3, 0, 0, 1,
        2, 2, 1, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 2, 0, 2, 0, 1, 0, 2, 2, 2,
        2, 0, 1, 2, 3, 2, 0, 1, 0, 2, 0,

In [21]:
predicted_labels = label_encoder.classes_[predicted_classes.numpy()]


In [28]:
predicted_labels

array(['Normal weight', 'Obese', 'Overweight', 'Obese', 'Normal weight',
       'Normal weight', 'Normal weight', 'Obese', 'Normal weight',
       'Normal weight', 'Normal weight', 'Normal weight', 'Overweight',
       'Normal weight', 'Obese', 'Normal weight', 'Normal weight',
       'Normal weight', 'Overweight', 'Overweight', 'Normal weight',
       'Normal weight', 'Obese', 'Normal weight', 'Underweight',
       'Normal weight', 'Normal weight', 'Obese', 'Overweight', 'Obese',
       'Overweight', 'Obese', 'Normal weight', 'Normal weight',
       'Overweight', 'Normal weight', 'Normal weight', 'Obese', 'Obese',
       'Obese', 'Underweight', 'Overweight', 'Obese', 'Normal weight',
       'Normal weight', 'Underweight', 'Obese', 'Normal weight',
       'Normal weight', 'Normal weight', 'Overweight', 'Normal weight',
       'Normal weight', 'Overweight', 'Normal weight', 'Underweight',
       'Overweight', 'Overweight', 'Overweight', 'Normal weight',
       'Overweight', 'Obese', 'Ov

In [22]:
X_with_predictions = X.copy()
X_with_predictions['Predicted_Obesity_Category'] = predicted_labels
X_with_predictions['Gender'] = gender_encoder.inverse_transform(X_with_predictions['Gender'])

In [23]:
X_with_predictions


,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,Predicted_Obesity_Category
0,56,Male,173.575262,71.982051,23.891783,4,Normal weight
1,69,Male,164.127306,89.959256,33.395209,2,Obese
2,46,Female,168.072202,72.930629,25.817737,4,Overweight
3,32,Male,168.459633,84.886912,29.912247,3,Obese
4,60,Male,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,Male,155.588674,64.103182,26.480345,4,Overweight
996,35,Female,165.076490,97.639771,35.830783,1,Obese
997,49,Female,156.570956,78.804284,32.146036,1,Obese
998,64,Male,164.192222,57.978115,21.505965,4,Normal weight
